In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, "C:/workspace/data-science//libs")
from big_numbers import contract_18_decimals_to_float

import warnings
warnings.filterwarnings('ignore')

In [2]:
tokens_addresses_to_names_dict = {
    "0x86f1e0420c26a858fc203a3645dd1a36868f18e5": "vBTC",
    "0x8c835dfaa34e2ae61775e80ee29e2c724c6ae2bb": "vETH",
    "0x5faa136fc58b6136ffdaeaac320076c4865c070f": "vAVAX",
    "0xb24f50dd9918934ab2228be7a097411ca28f6c14": "vLUNA",
    "0x151bb01c79f4516c233948d69dae39869bccb737": "vSOL",
    "0x7161c3416e08abaa5cd38e68d9a28e43a694e037": "vCRV",
    "0x333b1ea429a88d0dd48ce7c06c16609cd76f43a8": "vSAND",
    "0x2f198182ec54469195a4a06262a9431a42462373": "vLINK",
    "0x5f714b5347f0b5de9f9598e39840e176ce889b9c": "vATOM",
    "0x77d0cc9568605bfff32f918c8ffaa53f72901416": "vONE",
    "0x3fb3282e3ba34a0bff94845f1800eb93cc6850d4": "vNEAR",
    "0x2db8d2db86ca3a4c7040e778244451776570359b": "vFTM",
    "0x7eada83e15acd08d22ad85a1dce92e5a257acb92": "vFLOW",
    "0xb6599bd362120dc70d48409b8a08888807050700": "vBNB",
    "0x9482aafdced6b899626f465e1fa0cf1b1418d797": "vPERP",
    "0xbe5de48197fc974600929196239e264ecb703ee8": "vMATIC",
    "0x34235c8489b06482a99bb7fcab6d7c467b92d248": "vAAVE",
    "0x9d34f1d15c22e4c0924804e2a38cbe93dfb84bc2": "vAPE"
}

In [3]:
def liquidity_maker_individual_flow(liquidity_df: pd.DataFrame, position_changes_df: pd.DataFrame,
                                   maker: str, pool_name: str, xsize: int=15, ysize: int=14):
    maker_pool_liquidity_df = liquidity_df[(liquidity_df["pool_name"] == pool_name) &
                            (liquidity_df["maker"] == maker)]
    lower_bound_median = maker_pool_liquidity_df["lower_bound"].median()
    lower_bound_std = maker_pool_liquidity_df["lower_bound"].std()
    upper_bound_median = maker_pool_liquidity_df["upper_bound"].median()
    upper_bound_std = maker_pool_liquidity_df["upper_bound"].std()
    maker_pool_liquidity_df = maker_pool_liquidity_df[
        (maker_pool_liquidity_df["lower_bound"] > (lower_bound_median - 2 * lower_bound_std)) &
        (maker_pool_liquidity_df["upper_bound"] < (upper_bound_median + 2 * upper_bound_std))
    ]

    filtered_position_changes_df = position_changes_df[position_changes_df["pool_name"] == pool_name]
    price_median = filtered_position_changes_df["swapped_price"].median()
    price_std = filtered_position_changes_df["swapped_price"].std()
    filtered_position_changes_df = filtered_position_changes_df[
        (filtered_position_changes_df["swapped_price"] > (price_median - 2 * price_std)) &
        (filtered_position_changes_df["swapped_price"] < (price_median + 2 * price_std))
    ]

    #   bounds respective to the price
    fig = plt.figure(figsize=(15, 14))
    ax = fig.add_subplot(4, 1, 1)
    ax.plot(maker_pool_liquidity_df["time"], maker_pool_liquidity_df["lower_bound"], 
            color='r', label='lower bound')
    ax.plot(maker_pool_liquidity_df["time"], maker_pool_liquidity_df["upper_bound"], 
            color='g', label='upper bound')
    ax.plot(maker_pool_liquidity_df["time"], 
            (maker_pool_liquidity_df["upper_bound"] + maker_pool_liquidity_df["lower_bound"]) / 2, 
            color='b', linestyle='--', label='central bound')
    ax.plot(filtered_position_changes_df[
                (filtered_position_changes_df["timestamp"] > maker_pool_liquidity_df["timestamp"].min()) &
                (filtered_position_changes_df["timestamp"] < maker_pool_liquidity_df["timestamp"].max())
            ]["time"],
           filtered_position_changes_df[
                (filtered_position_changes_df["timestamp"] > maker_pool_liquidity_df["timestamp"].min()) &
                (filtered_position_changes_df["timestamp"] < maker_pool_liquidity_df["timestamp"].max())
            ]["swapped_price"],
           color='orange', label="swap price")
    ax.set_xlabel("time")
    ax.set_ylabel("USD")
    ax.legend()
    ax.grid()
    ax.set_title("Bounds estimated by " + maker + " for " + pool_name + " respective to the price for this token")

    #   fees and operations count distributions
    ax = fig.add_subplot(4, 1, 2)
    ax.plot(maker_pool_liquidity_df["time"], maker_pool_liquidity_df["quote_fee"], color='r',
            label='collected fees')

    ax2 = ax.twinx()
    count_changes_df = (pd.to_datetime(
        filtered_position_changes_df[
                (filtered_position_changes_df["timestamp"] > maker_pool_liquidity_df["timestamp"].min()) &
                (filtered_position_changes_df["timestamp"] < maker_pool_liquidity_df["timestamp"].max())
        ]['time']
    ).dt.floor('d').value_counts().rename_axis('date').reset_index(name='count'))
    count_changes_df.sort_values("date", inplace=True)
    ax2.plot(count_changes_df["date"], count_changes_df["count"], color='g', linestyle='--', label='count of swaps')
    ax.set_xlabel("time")
    ax.set_ylabel("USD")
    ax2.set_ylabel("operations count")
    ax.grid()
    ax.set_title("Collected fees and count of operations in " + pool_name +" by " + maker)

    # demonstration of the provided and extracted liquidities
    ax = fig.add_subplot(4, 1, 3)
    ax.plot(maker_pool_liquidity_df[maker_pool_liquidity_df["fixed_liquidity"] >= 0]["time"],
           maker_pool_liquidity_df[maker_pool_liquidity_df["fixed_liquidity"] >= 0]["fixed_liquidity"],
           color='g', label='investments')
    ax.plot(maker_pool_liquidity_df[maker_pool_liquidity_df["fixed_liquidity"] < 0]["time"],
           maker_pool_liquidity_df[maker_pool_liquidity_df["fixed_liquidity"] < 0]["fixed_liquidity"],
           color='r', label='extractions')
    ax.set_xlabel("time")
    ax.set_ylabel("USD")
    ax.set_title("Provided and extracted liquidities from the " + pool_name + " by " + maker)
    ax.grid(True, linestyle='--')

    #   liquidity balance
    ax = fig.add_subplot(4, 1, 4)
    maker_pool_liquidity_df["liquidity_balance"] = maker_pool_liquidity_df["fixed_liquidity"].cumsum()
    ax.plot(maker_pool_liquidity_df["time"], maker_pool_liquidity_df["liquidity_balance"],
           color='b', label='balance')
    ax.set_xlabel("time")
    ax.set_ylabel("USD")
    ax.set_title("Balance of the provided liquidity to the " + pool_name + " by " + maker)
    ax.grid(True, linestyle='--')
    fig.tight_layout()
    plt.show()

# Loading liquidity

In [13]:
liquidity_changes_df = pd.read_csv("liquidity_changes.csv")
liquidity_changes_df["time"] = pd.to_datetime(liquidity_changes_df["timestamp"], unit='s')
liquidity_changes_df["day_of_year"] = liquidity_changes_df["time"].dt.dayofyear
liquidity_changes_df["pool_name"] = liquidity_changes_df["base_token"].replace(tokens_addresses_to_names_dict)
liquidity_changes_df.drop(columns=["base_token", "tx_hash", "from_function_signature"], inplace=True)
liquidity_changes_df["lower_bound"] = pow(1.0001, liquidity_changes_df["lower_tick"])
liquidity_changes_df["upper_bound"] = pow(1.0001, liquidity_changes_df["upper_tick"])
liquidity_changes_df["fixed_liquidity"] = [contract_18_decimals_to_float(liquidity_record) for 
                                           liquidity_record in liquidity_changes_df["liquidity"]]
liquidity_changes_df.drop(columns=["Unnamed: 0", "quote_token", "lower_tick", "upper_tick", "liquidity", "block_number"], 
                          inplace=True)
liquidity_changes_df["central_bound"] = (liquidity_changes_df["upper_bound"] + liquidity_changes_df["lower_bound"]) / 2
liquidity_changes_df.sort_values("block_number_log_index", inplace=True)
liquidity_changes_df = liquidity_changes_df.reset_index()
liquidity_changes_df.drop(columns=["index"], inplace=True)